<a href="https://colab.research.google.com/github/antonioRVR/SMS_text_Ham_Spam_clasiffication/blob/main/sms_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

2.15.0


We get the data files

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

--2024-01-24 11:38:58--  https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 104.26.3.33, 172.67.70.149, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 358233 (350K) [text/tab-separated-values]
Saving to: ‘train-data.tsv.1’

train-data.tsv.1    100%[===================>] 349.84K  --.-KB/s    in 0.03s   

2024-01-24 11:38:59 (11.0 MB/s) - ‘train-data.tsv.1’ saved [358233/358233]

--2024-01-24 11:38:59--  https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 104.26.3.33, 172.67.70.149, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 118774 (116K) [text/tab-separated-values]
Saving to: ‘valid-data.tsv.1’

valid-data.tsv.1    100%[========

https://colab.research.google.com/drive/1ysEKrw_LE2jMndo1snrZUh5w87LQsCxk#forceEdit=true&sandboxMode=true

#Add code from here ⛳

We extract the data

In [ ]:
train=pd.read_csv(train_file_path,sep='\t', names=["A","B"])
test=pd.read_csv(test_file_path,sep='\t', names=["A","B"])

In [ ]:
train.shape
test.shape

(1392, 2)

In [ ]:
type(train)
type(test)

pandas.core.frame.DataFrame

In [ ]:
train.head()
test.head()

,A,B
0,ham,i am in hospital da. . i will return home in e...
1,ham,"not much, just some textin'. how bout you?"
2,ham,i probably won't eat at all today. i think i'm...
3,ham,don‘t give a flying monkeys wot they think and...
4,ham,who are you seeing?


In [ ]:
train= train.to_numpy()
test= test.to_numpy()

In [ ]:
print(type(train))
print(type(test))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [ ]:
train_labels = train[:, 0]
train_data = train[:, 1]
test_labels = test[:, 0]
test_data = test[:, 1]

In [ ]:
print(train.shape)
print(test.shape)
print(train_labels.shape)
print(test_labels.shape)
print(train_data.shape)
print(test_data.shape)

(4179, 2)
(1392, 2)
(4179,)
(1392,)
(4179,)
(1392,)


In [ ]:
# Display the first few rows of each array
print("Training Data:")
print(train_data[:5])

print("\nValidation Data:")
print(test_data[:5])

print("Training labels:")
print(train_labels[:5])

print("\nValidation labels:")
print(test_labels[:5])

Training Data:
['ahhhh...just woken up!had a bad dream about u tho,so i dont like u right now :) i didnt know anything about comedy night but i guess im up for it.'
 'you can never do nothing'
 'now u sound like manky scouse boy steve,like! i is travelling on da bus home.wot has u inmind 4 recreation dis eve?'
 'mum say we wan to go then go... then she can shun bian watch da glass exhibition...'
 'never y lei... i v lazy... got wat? dat day ü send me da url cant work one...']

Validation Data:
['i am in hospital da. . i will return home in evening'
 "not much, just some textin'. how bout you?"
 "i probably won't eat at all today. i think i'm gonna pop. how was your weekend? did u miss me?"
 'don‘t give a flying monkeys wot they think and i certainly don‘t mind. any friend of mine and all that!'
 'who are you seeing?']
Training labels:
['ham' 'ham' 'ham' 'ham' 'ham']

Validation labels:
['ham' 'ham' 'ham' 'ham' 'ham']


In [ ]:
train_data[1]

'you can never do nothing'

In [ ]:
train_labels = np.where(train_labels == "ham", 0, 1)
test_labels = np.where(test_labels == "ham", 0, 1)

In [ ]:
test_labels[1]

0

Now we've got correctly divided our data and its labels

## More Processesing

we have a look at some of our loaded in reviews, we'll notice that they are different lengths. This is an issue. We cannot pass different length data into our neural network. Therefore, we must make each review the same length. To do this we will follow the procedure below:

Max of words



In [ ]:
def contar_palabras(cadena):
    return len(cadena.split())

# Aplicar la función a cada elemento del array
num_palabras_por_elemento = np.vectorize(contar_palabras)(train_data)

# Encontrar el índice del elemento con la máxima cantidad de palabras
indice_max_palabras = np.unravel_index(np.argmax(num_palabras_por_elemento), num_palabras_por_elemento.shape)

# Obtener la cantidad de palabras y la cadena con la mayor cantidad de palabras
cantidad_max_palabras = num_palabras_por_elemento[indice_max_palabras]
cadena_max_palabras = train[indice_max_palabras]

# Imprimir el resultado
print("\nNúmero de palabras por elemento para train:")
print(num_palabras_por_elemento)
print("\nElemento con la mayor cantidad de palabras:", indice_max_palabras)
print(f"Elemento: {cadena_max_palabras}, Cantidad de palabras: {cantidad_max_palabras}")


Número de palabras por elemento para train:
[30  5 22 ... 18 24 32]

Elemento con la mayor cantidad de palabras: (1139,)
Elemento: ['ham'
 "for me the love should start with attraction.i should feel that i need her every time around me.she should be the first thing which comes in my thoughts.i would start the day and end it with her.she should be there every time i dream.love will be then when my every breath has her name.my life should happen around her.my life will be named to her.i would cry for her.will give all my happiness and take all her sorrows.i will be ready to fight with anyone for her.i will be in love when i will be doing the craziest things for her.love will be when i don't have to proove anyone that my girl is the most beautiful lady on the whole planet.i will always be singing praises for her.love will be when i start up making chicken curry and end up makiing sambar.life will be the most beautiful then.will get every morning and thank god for the day because she is w

##Encoding

We create the function for the encoding

In [ ]:
all_text=result = np.concatenate((train_data, test_data), axis=0)

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

# Create a tokenizer and fit on the texts
tokenizer = Tokenizer()
tokenizer.fit_on_texts(all_text)

# Convert the texts to integer sequences
train_data_tokenized = np.array(tokenizer.texts_to_sequences(train_data))
test_data_tokenized= np.array(tokenizer.texts_to_sequences(test_data))

VOCAB_SIZE = len(tokenizer.word_index)

print("Tamaño del Vocabulario:", VOCAB_SIZE)

Tamaño del Vocabulario: 8995


<ipython-input-64-bb6def07d81f>:9: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  train_data_tokenized = np.array(tokenizer.texts_to_sequences(train_data))
<ipython-input-64-bb6def07d81f>:10: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  test_data_tokenized= np.array(tokenizer.texts_to_sequences(test_data))


In [ ]:
from keras.preprocessing import sequence
MAXLEN=cantidad_max_palabras
print(MAXLEN)

171


In [ ]:
train_data_padden = sequence.pad_sequences(train_data_tokenized, MAXLEN)
test_data_padden = sequence.pad_sequences(test_data_tokenized, MAXLEN)

In [ ]:
train_data_tokenized.shape

(4179,)

In [ ]:
test_data_padden.shape

(1392, 171)

##Creation of the Model

Change to GPU in order to avoid problem on building the model

In [ ]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense
import keras
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, 32),
    tf.keras.layers.LSTM(32),
    tf.keras.layers.Dense(1, activation="sigmoid")
])

# Compilar el modelo
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Obtener el resumen del modelo después de compilarlo
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, None, 32)          287840    
                                                                 
 lstm_2 (LSTM)               (None, 32)                8320      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 296193 (1.13 MB)
Trainable params: 296193 (1.13 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
print(type(train_data_tokenized))

<class 'numpy.ndarray'>


In [ ]:
model.compile(loss="binary_crossentropy",optimizer="rmsprop",metrics=['acc'])

history = model.fit(train_data_padden, train_labels, epochs=10, validation_split=0.2)

Epoch 1/10
105/105 [==============================] - 21s 169ms/step - loss: 0.2737 - acc: 0.9061 - val_loss: 0.1219 - val_acc: 0.9653
Epoch 2/10
105/105 [==============================] - 4s 41ms/step - loss: 0.0839 - acc: 0.9791 - val_loss: 0.0613 - val_acc: 0.9856
Epoch 3/10
105/105 [==============================] - 4s 43ms/step - loss: 0.0429 - acc: 0.9892 - val_loss: 0.0460 - val_acc: 0.9880
Epoch 4/10
105/105 [==============================] - 3s 24ms/step - loss: 0.0315 - acc: 0.9919 - val_loss: 0.0622 - val_acc: 0.9856
Epoch 5/10
105/105 [==============================] - 2s 22ms/step - loss: 0.0216 - acc: 0.9943 - val_loss: 0.0457 - val_acc: 0.9892
Epoch 6/10
105/105 [==============================] - 1s 14ms/step - loss: 0.0183 - acc: 0.9955 - val_loss: 0.0471 - val_acc: 0.9880
Epoch 7/10
105/105 [==============================] - 2s 16ms/step - loss: 0.0121 - acc: 0.9967 - val_loss: 0.0577 - val_acc: 0.9844
Epoch 8/10
105/105 [==============================] - 2s 16ms/step 

In [ ]:
results = model.evaluate(test_data_padden, test_labels)
print(results)

44/44 [==============================] - 0s 7ms/step - loss: 0.0665 - acc: 0.9835
[0.06647258251905441, 0.983476996421814]


You should create a function called predict_message taht takes a message string as an argument and returns a list. The first element in the list should be a number between zero and one that indicates the likeliness of "ham" (0) or "spam" (1). The second element in the list should be the word "ham" or "spam", depending on which is most likely

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
    # Tokeniza el texto
    pred_text_tokenized = tokenizer.texts_to_sequences([pred_text])
    # Realiza el relleno o truncado para que tenga la misma longitud que el modelo espera
    pred_text_padded = pad_sequences(pred_text_tokenized, maxlen=MAXLEN)
    # Realiza la predicción
    prediction_prob = model.predict(pred_text_padded)[0][0]
    # Asigna la etiqueta
    label = 'ham' if prediction_prob < 0.5 else 'spam'
    # Devuelve la lista con la predicción y la etiqueta
    prediction=[prediction_prob, label]


    return (prediction)

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

1/1 [==============================] - 0s 387ms/step
[0.00023335978, 'ham']


tests your model and function.

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()


1/1 [==============================] - 0s 28ms/step
You passed the challenge. Great job!
